# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [22]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.model_selection import GridSearchCV

In [5]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [9]:
spaceship.dropna(inplace=True)
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x[0])
numbers = spaceship.select_dtypes("number")
categories = spaceship.select_dtypes("object")
normalizer = MinMaxScaler()
normalizer.fit(numbers)
numbers_norm = normalizer.transform(numbers)
numbers_norm =  pd.DataFrame(numbers_norm, columns = numbers.columns)
cat_dummies = pd.get_dummies(categories)
df = numbers.join(cat_dummies).join(spaceship["Transported"])



- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [17]:


features = df.drop(columns = ["Transported"])
target = df["Transported"]

X_train, X_test, y_train, y_test = train_test_split(features,target, test_size=0.20, random_state=0)


bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100, # number of models to use
                               max_samples = 1000)
bagging_reg.fit(X_train, y_train)

normalizer = MinMaxScaler()
normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns, index=X_train.index)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns, index=X_test.index)


- Evaluate your model

In [18]:
# bagging and Pasting result


y_pred_test_bag = bagging_reg.predict(X_test)

print(f"MAE {mean_absolute_error(y_pred_test_bag, y_test): .2f}")
print(f"RMSE {mean_squared_error(y_pred_test_bag, y_test, squared=False): .2f}")
print(f"R2 score {bagging_reg.score(X_test_norm, y_test): .2f}")

MAE  0.25
RMSE  0.38


/home/tiago/anaconda3/envs/conda_base/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


R2 score -0.78


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [20]:
grid = {"max_leaf_nodes": [250, 500, 1000],
        "max_depth":[10,30,50]}
dt = DecisionTreeRegressor()

- Run Grid Search

In [23]:
model = GridSearchCV(estimator = dt, param_grid = grid, cv=5, verbose=10) # The "cv" option here is used to provide the desired number of folds K.
model.fit(X_train_norm, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 1/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.408 total time=   2.4s
[CV 2/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 2/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.374 total time=   2.1s
[CV 3/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 3/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.335 total time=   2.1s
[CV 4/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 4/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.325 total time=   2.2s
[CV 5/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 5/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.380 total time=   2.1s
[CV 1/5; 2/9] START max_depth=10, max_leaf_nodes=500............................
[CV 1/5; 2/9] END max_depth=10, ma

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [10, 30, 50],
                         'max_leaf_nodes': [250, 500, 1000]},
             verbose=10)

- Evaluate your model

In [24]:
model.best_params_

{'max_depth': 10, 'max_leaf_nodes': 250}

In [25]:
best_model = model.best_estimator_

In [26]:
pred = best_model.predict(X_test_norm)

print(f"MAE: {mean_absolute_error(pred, y_test): .2f}")
print(f"RMSE: {mean_squared_error(pred, y_test, squared=False): .2f}")
print(f"R2 score:  {best_model.score(X_test_norm, y_test): .2f}")

MAE:  0.27
RMSE:  0.41
R2 score:   0.33


/home/tiago/anaconda3/envs/conda_base/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
